In [1]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import json
from transformers import AutoTokenizer, BertModel
import tqdm
import torch.nn as nn
import math

# custom dataset class to load our data

class OurDataset(Dataset):
    def __init__(self, data_file, labels_file):
        self.full_data = json.load(open(data_file))
        self.labels = torch.load(labels_file)

        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        self.model = BertModel.from_pretrained("bert-base-uncased")
        
    def __len__(self):
        return len(self.full_data) 
    
    def __getitem__(self, idx):
        inputs = self.tokenizer(self.full_data[idx], return_tensors="pt")
        outputs = self.model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        return last_hidden_states, self.labels[idx]

/Users/anonymous/miniforge3/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [2]:
import concurrent.futures
import json
import os
import random
from datetime import datetime
import time

import numpy as np

import openai
openai.api_key = ""

class EvoPrompting:
    def __init__(self, T, M, K, seed_folder, prepend_file, param_threshold):
        self.T = T  #how many total evolution steps to go through
        self.M = M  #how many children to generate in each evolution step
        self.K = K  #how many children are allowed to survive in each evolution step
        self.seed_folder = seed_folder
        self.prepend_file = prepend_file
        self.param_threshold = param_threshold
        self.population = []
        self.ds = OurDataset("data/full_data.json", "data/labels.torch")
        self.curr_time = datetime.now()
        self.initialize_population()
        
    def initialize_population(self):
        def read_seed_files(file_path):
            with open(file_path, "r") as file:
                return file.read()
        
        self.prepend_code = read_seed_files(self.prepend_file)
        seed_files = [f for f in os.listdir(self.seed_folder) if f.endswith('.py')]
        for seed_file in seed_files:
            seed_file_path = os.path.join(self.seed_folder, seed_file)
            seed_code = read_seed_files(seed_file_path)
            model, metrics = self.exec_code(seed_code)
            if (model, metrics) == (0, 0):
                continue
            self.population.append((seed_code, model, metrics))
        
    def exec_code(self, code):
        def single_evaluation():
            print("Executing code segment")
            exec(self.prepend_code, globals())
            exec(code, globals())
            model = globals()['main']()
            return model
        def count_parameters(model):
            return sum(p.numel() for p in model.parameters() if p.requires_grad)
        
        model = single_evaluation()
        if count_parameters(model) >= self.param_threshold:
            return (0, 0)
        metrics = self.eval_model(model)
        print("No. of Parameters: ", count_parameters(model))
        print("Model Score: ", metrics)
        return model, metrics
    
    #evaluate model without training here (just a classification forward pass)
    def eval_model(self, model):
        loss_fn = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=0.00001, momentum=0.9)
        loss = None
        for i in range(50):
            idx = random.randint(0, self.ds.__len__()-3000)
            x, y = self.ds.__getitem__(idx)
            y = y.long()
            outputs = model(x)
            loss = loss_fn(outputs, y)
            loss.backward()
            optimizer.step()
        correct = 0
        for i in range(50):
            idx = random.randint(0, self.ds.__len__()-3000)
            x, y = self.ds.__getitem__(idx)
            y = y.long()
            outputs = model(x)
            outputs = torch.argmax(outputs, axis=-1)
            if outputs == y:
                correct += 1
        return correct/50
    
    def evaluate_children(self, child_architectures):
        for code in child_architectures:
            try:
                model, metrics = self.exec_code(code)
                self.population.append((code, model, metrics))
            except:
                continue
    
    # setup prompt and generate children
    def cross_mutation(self):
        while (datetime.now() - self.curr_time).seconds <= 60:
            time.sleep(2)
        self.curr_time = datetime.now()
        child_architectures = []
        for m in range(self.M):
            prompt = ""
            for arch in self.population: 
                prompt += "Code: " + arch[0] + "\n\n"
            prompt += "Code:"
            #print(prompt)
            messages = [#{"role":"system", "content":"You are a helpful assistant whose job is to generate better models based on the example given to you. Only generate valid python code."},
                       {"role":"user", "content":prompt}]
            response = openai.Completion.create(model="gpt-3.5-turbo-instruct", prompt=prompt, n=1, max_tokens=1600, temperature=0)
            #response = client.chat.completions.create(model="gpt-3.5-turbo-instruct", messages=messages, n=1, max_tokens=1600, temperature=0)
            # print(response["choices"][0]["text"].lstrip())
            child_architectures.append(response["choices"][0]["text"].lstrip())
        return child_architectures
    
    def cull_population(self):
        self.population = sorted(self.population, key=lambda x: x[-1], reverse=True)[:self.K]
        
    def evolve(self):
        for t in range(self.T):
            child_architectures = self.cross_mutation()
            self.evaluate_children(child_architectures)
            self.cull_population()
            print("Current Population Scores: ", [p[-1] for p in self.population])
        return self.population

In [3]:
evo = EvoPrompting(T=4, M=3, K=3, seed_folder="./seeds", prepend_file="./prepend.py", param_threshold=20000000)

Executing code segment


/Users/anonymous/miniforge3/lib/python3.9/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


No. of Parameters:  12614915
Model Score:  0.38
Executing code segment
No. of Parameters:  6112259
Model Score:  0.36
Executing code segment
No. of Parameters:  11041027
Model Score:  0.42
Executing code segment
No. of Parameters:  11631619
Model Score:  0.4
Executing code segment
No. of Parameters:  11631619
Model Score:  0.32
Executing code segment
No. of Parameters:  6112259
Model Score:  0.4
Executing code segment
No. of Parameters:  12614915
Model Score:  0.42


In [4]:
population = evo.evolve()

Executing code segment
Executing code segment
Executing code segment
Current Population Scores:  [0.42, 0.42, 0.4]
Executing code segment
No. of Parameters:  12222211
Model Score:  0.34
Executing code segment
No. of Parameters:  12222211
Model Score:  0.4
Executing code segment
No. of Parameters:  12222211
Model Score:  0.42
Current Population Scores:  [0.42, 0.42, 0.42]
Executing code segment
No. of Parameters:  12222211
Model Score:  0.44
Executing code segment
No. of Parameters:  12222211
Model Score:  0.46
Executing code segment
No. of Parameters:  12222211
Model Score:  0.44
Current Population Scores:  [0.46, 0.44, 0.44]
Executing code segment
No. of Parameters:  12222211
Model Score:  0.38
Executing code segment
No. of Parameters:  12222211
Model Score:  0.32
Executing code segment
No. of Parameters:  12222211
Model Score:  0.58
Current Population Scores:  [0.58, 0.46, 0.44]


In [5]:
for p in population:
    print(p[0])

class TransformerClassifier(nn.Module):

    def __init__(
        self,
        d_model=768,
        n_classes=3,
        nhead=4,
        dim_feedforward=256,
        num_layers=4,
        dropout=0.1,
        activation="relu",
        classifier_dropout=0.1,
    ):

        super().__init__()

        self.pos_encoder = PositionalEncoding(
            d_model=d_model,
            dropout=dropout,
            max_len=5000,
        )

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers,
        )
        self.classifier1 = nn.Linear(d_model, d_model)
        self.classifier2 = nn.Linear(d_model, d_model)
        self.classifier3 = nn.Linear(d_model, n_classes)
        self.softmax = nn.Softmax(dim=1)
        self.d_model = d_model


In [68]:
from tests.genbig import TransformerClassifier
t = TransformerClassifier()
ds = OurDataset("data/full_data.json", "data/labels.torch")

In [69]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(t.parameters(), lr=0.00001, momentum=0.9)
loss = None
for i in range(ds.__len__()):
    idx = random.randint(0, ds.__len__()-1)
    x, y = ds.__getitem__(idx)
    y = y.long()
    outputs = t(x)
    loss = loss_fn(outputs, y)
    if i%10 == 0:
        print(i, loss.item())
    loss.backward()
    optimizer.step()
    
correct = 0
true = []
pred = []
for i in range(1000):
    idx = random.randint(0, ds.__len__()-1)
    x, y = ds.__getitem__(idx)
    y = y.long()
    outputs = t(x)
    outputs = torch.argmax(outputs, axis=-1)
    true.append(y.item())
    pred.append(outputs.item())

0 0.9759292602539062
10 1.1690107583999634
20 1.1952286958694458
30 1.0678808689117432
40 1.2093340158462524
50 1.2363804578781128
60 0.7992780804634094
70 1.4137789011001587
80 0.6471167802810669
90 0.6237916946411133
100 0.5968904495239258
110 0.5764997601509094
120 0.5660012364387512
130 1.5397109985351562
140 0.5550039410591125
150 0.5527303814888
160 0.5520373582839966
170 0.5517510771751404
180 0.5515452027320862
190 0.5515131950378418
200 0.551461935043335
210 0.5514549016952515
220 0.5514509677886963
230 1.5514402389526367
240 0.5514451861381531
250 0.5514450669288635
260 0.5514447689056396
270 0.5514447689056396
280 0.5514447093009949
290 1.5514447689056396
300 0.5514447093009949
310 0.5514447093009949
320 0.5514447093009949
330 1.5514447689056396
340 0.5514447093009949
350 0.5514447093009949
360 1.5514447689056396
370 1.5514447689056396
380 1.5514447689056396
390 0.5514447093009949
400 1.5514447689056396
410 0.5514447093009949
420 1.5514447689056396
430 1.5514447689056396
440

In [70]:
from sklearn.metrics import f1_score

print("F1 Score (weighted): ", f1_score(true, pred, average="weighted"))
print("F1 Score (macro): ", f1_score(true, pred, average="macro"))
print("F1 Score (micro): ", f1_score(true, pred, average="micro"))

F1 Score (weighted):  0.5146537530266344
F1 Score (macro):  0.2631154156577885
F1 Score (micro):  0.652
